# Chapter 10: Mutual information

As we saw [in class](https://mml.johnmyersmath.com/stats-book/chapters/10-info-theory.html#flow-of-information), the technical notion of _mutual information_ captures much of what we intuitively understand a measure of "transferred information" should be. In this programming assignment, we will work though an extended example involving the transmission of bit sequences through a _binary symmetric channel_ in order to help further develop our intuition for _mutual information_.

## Directions

1. The programming assignment is organized into sequences of short problems. You can see the structure of the programming assignment by opening the "Table of Contents" along the left side of the notebook (if you are using Google Colab or Jupyter Lab).

2. Do not add any cells of your own to the notebook, or delete any existing cells (either code or markdown).

## Submission instructions

1. Once you have finished entering all your solutions, you will want to rerun all cells from scratch to ensure that everything works OK. To do this in Google Colab, click "Runtime -> Restart and run all" along the top of the notebook.

2. Now scroll back through your notebook and make sure that all code cells ran properly.

3. If everything looks OK, save your assignment and upload the `.ipynb` file at the provided link on the course <a href="https://github.com/jmyers7/stats-book-materials">GitHub repo</a>. Late submissions are not accepted.

4. You may submit multiple times, but I will only grade your last submission.

## Modeling communication channels

Suppose that a friend is attempting to transmit a message to you through some sort of communication channel that is prone to error. Perhaps the simplest situation is to imagine that your friend's message is an encoded sequences of bits ($0$'s and $1$'s) and that the channel will commit errors by randomly flipping some of the bits---a $0$ randomly flips to a $1$, and a $1$ randomly flips to a $0$.

How might we model this situation mathematically?

Recall from class that our precise definition of a _communication channel_ is a _Markov kernel_, meaning a function

$$
\kappa:\{1,2,\ldots,m\} \to \mathbb{R}^n
$$

such that each vector $\kappa(i)$ is a _probability vector_, i.e., a vector with nonnegative entries that sum to $1$. A Markov kernel may be conveniently described via its _transition matrix_, which is an $m\times n$ matrix with the vectors $\kappa(i)$ slotted into the rows:

$$
\mathbf{K} = \begin{bmatrix}
\leftarrow & \kappa(1)^\intercal & \rightarrow \\
\vdots & \vdots & \vdots \\
\leftarrow & \kappa(m)^\intercal & \rightarrow
\end{bmatrix}.
$$

Your friend (the sender) generates a message consisting of a sequence $x_1,x_2,\ldots$ of bits, transmits them through the channel, and you (the receiver) see $y_1,y_2,\ldots$. The standard diagram of the situation is this:

<br>
<center>
<img src="https://raw.githubusercontent.com/jmyers7/stats-book-materials/main/img/channel-01.svg?raw=true" width="500" align="center">
</center>
<br>

But because the channel is "noisy" and introduces errors, your received bit sequence $y_1,y_2,\ldots$ might not match the original bit sequence $x_1,x_2,\ldots$.

The input bit sequence is modeled as a sequence of independent observations of a "sender" random variable $X$, and the output bits are modeled as observations of a "receiver" random variable $Y$:

<br>
<center>
<img src="https://raw.githubusercontent.com/jmyers7/stats-book-materials/main/img/channel.svg?raw=true" width="500" align="center">
</center>
<br>

For simplicitly, let's suppose that the communication channel is given by the _binary symmetric channel_ that we saw briefly in class, with transition matrix

$$
\mathbf{K} = \begin{bmatrix}
1 - \phi & \phi \\
\phi & 1-\phi
\end{bmatrix},
$$

for some parameter $\phi \in [0,1]$. The entries in this matrix are then conceptualized as conditional probabilities:

$$
\begin{array}{c|cc}
p(y|x) & y=0 & y=1 \\ \hline
x=0 & 1 - \phi & \phi \\
x=1 & \phi & 1 - \phi \\
\end{array}
$$

We think of these probabilities as follows: Since

$$
p(y=0|x=1) = p(y=1|x=0) = \phi,
$$

the probability that the channel flips the bit is $\phi$, while since

$$
p(y=0|x=0) = p(y=1|x=1) = 1-\phi,
$$

the probability that the bit is received without error is $1-\phi$. For this reason, the parameter $\phi \in [0,1]$ is called the _flip probability_.


## Problem 1 --- Simulating input sequences

Because the random variable $X$ takes values in $\{0,1\}$, it is a Bernoulli random variable, say $X\sim Ber(\alpha)$ for some $\alpha\in [0,1]$. In the next code cell, use the given value of $\alpha$ and implement a Bernoulli random variable called `X` using SciPy. I've written template code for you; all you need to do is replace the `None` placeholder with your own code.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

import scipy as sp
import numpy as np

alpha = 0.35
X = None          # <-- replace `None` with your own code

Now, in the next code cell, simulate an input sequence of length $100$ called `input` using your random variable `X`. As above, I've written the template for you---just replace the `None` placeholder.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

np.random.seed(42)
input = None        # <-- replace `None` with your own code
print(input)

## Problem 2 --- Simulating the binary symmetric channel

How can we model the "flipping of a bit"? One clever way consists of the following: Suppose the input bit is $x \in \{0,1\}$. Then, obviously we have

$$
x + 0 \ (\text{mod $2$}) = x,
$$

while

$$
x + 1 \ (\text{mod $2$}) = \begin{cases}
1 & : x=0, \\
0 & : x=1.
\end{cases}
$$

This shows that adding $1$ (modulo $2$) has the effect of flipping a bit, while adding $0$ (modulo _anything_) has the effect of preserving a bit.

So, we may simulate the "flipping of a bit with probability $\phi$" by first sampling a draw $f$ from a Bernoulli random variable $F\sim Ber(\phi)$, and then computing

$$
y = x + f \ (\text{mod $2$}).
$$

With this in mind, I want you to define a function called `transmit` that simulates the binary symmetric channel. It will take two parameters: The first is `input`, a NumPy array containing the input sequence, and the second is `phi`, the flip probability. I have given you the template for the `transmit` function in the next code cell. Your job is to replace the `None` placeholders with appropriate code to implement the "flipping of a bit" described above. (You'll have to look up how to do modular arithmetic in Python.)


In [ ]:
# ENTER YOUR CODE IN THIS CELL

def transmit(input, phi):
  f = None                  # <-- replace `None` with your own code
  output = None             # <-- replace `None` with your own code
  return output

## Problem 3 --- Simulating output sequences

Now that we've implemented the input random variable and the communication channel, we may simulate output sequences. In the next code cell, use the input sequence stored in the variable `input` from Problem 1 and the given value of $\phi$ to produce an output sequence using the function `transmit`. Store the result in the variable `output`.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

phi = 0.65
output = None       # <-- replace `None` with your own code
print(output)

Since the output random variable $Y$ takes values in $\{0,1\}$, it too is a Bernoulli random variable, say $Y \sim Ber(\beta)$ for some parameter $\beta \in [0,1]$. Given the output bit sequence stored in `output`, in the next cell find an estimate for the probability $\beta$ of observing a $1$.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

beta = None       # <-- replace `None` with your own code
print(beta)

The value of $\beta$ you just computed is an _empirical estimate_. However, in class we learned a formula for the _exact_ value of $\beta$. In the next code cell, enter this exact value, given the values of $\alpha$ and $\phi$ fixed above.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

beta = None     # <-- replace `None` with your own code
print(beta)

## Measuring transmitted information

We measure the information passed through the channel as the mutual information $I(X,Y)$. Our goal now is to learn how this information depends on the two parameters $\alpha$ and $\phi$.

The formula for the (theoretical) mutual information from class is

$$
I(X,Y) = D(P_{XY} \parallel P_X \otimes P_Y ) = \sum_{(x,y) \in \mathbb{R}^2} p(x,y) \log_2 \left( \frac{p(x,y)}{p(x)p(y)} \right).
$$

While it _is_ possible to write down an explict formula for $I(X,Y)$, we shall go down a different road and compute sample-based _empirical mutual information_. This just means that we replace the joint mass $p(x,y)$ and the two marginal masses $p(x)$ and $p(y)$ with empirical distributions induced by an observed sequence of bits

$$
(x_1,y_1),(x_2,y_2),\ldots, (x_n,y_n),
$$

where the $x$'s are input bits and the $y$'s are output bits.

This is a little tricky for new coders, so I wrote a function for you, called `mutual_information`, that computes these (empirical) mutual informations. Run the following cell:

In [ ]:
def mutual_information(input, output):
  n0 = np.count_nonzero(input == 0)
  n1 = np.count_nonzero(input == 1)
  px = np.array([n0, n1]) / len(input)

  n0 = np.count_nonzero(output == 0)
  n1 = np.count_nonzero(output == 1)
  py = np.array([n0, n1]) / len(output)

  p00 = sum((input == 0) & (output == 0)) / len(input)
  p01 = sum((input == 0) & (output == 1)) / len(input)
  p10 = sum((input == 1) & (output == 0)) / len(input)
  p11 = sum((input == 1) & (output == 1)) / len(input)
  pxy = np.array([[p00, p01], [p10, p11]])

  information = sp.stats.entropy(pk=pxy, qk=np.outer(px, py), base=2, axis=None)
  return information

Let's see if this function works by testing it on our input and output sequences generated above. Run the following cell:

In [ ]:
mutual_information(input, output)

## Problem 4 --- Mutual information and input sequences

Now we wonder: For a fixed flip probability $\phi$, how does the mutual information $I(X,Y)$ depend on the parameter $\alpha$ in $X \sim Ber(\alpha)$? Let's produce a plot to find out.

For each $\alpha$ in a range of values from $0$ to $1$, the following code randomly generates an input sequence drawn from $X\sim Ber(\alpha)$, transmits it, and then computes the mutual information. As $\alpha$ ranges through the loop, it collects these mutual informations in a running list, and then plots them. Your job is to fill in the `None` placeholders with appropriate code to accomplish this task.

In [ ]:
# ENTER YOUR CODE IN THIS CELL

import matplotlib.pyplot as plt

phi = 0.65
step = 0.005
alphas = np.arange(0, 1 + step, step)
info_list = []
np.random.seed(42)

for alpha in alphas:
  X = None                # <-- replace `None` with your own code
  input = None            # <-- replace `None` with your own code
  output = None           # <-- replace `None` with your own code
  info = None             # <-- replace `None` with your own code
  info_list.append(info)

plt.plot(alphas, info_list)
plt.xlabel('$\\alpha$')
plt.ylabel('mutual information')
plt.show()

Interesting. The plot is spiky because the mutual informations were computed through generating random input sequences, but it does appear that the mutual information $I(X,Y)$ is maximized around $\alpha \approx 0.5$, and minimized as $\alpha$ nears $0$ and $1$. Try changing the flip probability $\phi$ to different values and run the cell again---what do you see? Does something funny happen around $\phi \approx 0.5$? Can you come up with an explanation for why these plots have these shapes? I encourage you think about this on your own for a little while, to help develop your intuition for mutual information.

## Problem 5 --- Mutual information and flip probability

But now, let's turn our attention toward the effect that the flip probability $\phi$ has on the mutual information. In the next code cell, produce a plot of the mutual information $I(X,Y)$ versus $\phi$. (_Hint_: Copy and paste the code from the plot above, and just make the "obvious" changes.)

In [ ]:
# ENTER YOUR CODE IN THIS CELL



Does the plot make intuitive sense to you? For what particular values of $\phi$ does the mutual information $I(X,Y)$ appear to be minimized or maximized?

## Wrap-up

In his foundational paper ([cited in the book](https://mml.johnmyersmath.com/stats-book/chapters/bib.html)), Shannon defined the _channel capacity_ of a communication channel to be the maximum mutual information $I(X,Y)$ over _all_ possible probability distributions on the input random variable $X$. In the special case of the binary symmetric channel with a fixed flip probability $\phi$, this capacity is therefore

$$
\text{channel capacity} = \max_{\alpha \in [0,1]} I(X,Y),
$$

where $X\sim Ber(\alpha)$. So, the plots generated in Problem 4 essentially identify the channel capacity to be the mutual information $I(X,Y)$ when $\alpha \approx 0.5$. In fact, the channel capacity occurs _exactly_ at $\alpha=0.5$, which is not difficult to prove. But the point is that we discovered the channel capacity _empirically_ through simulations. This nicely demonstrates a pattern of experimental results suggesting theoretical ones.

The channel capacity is a fundamental quantity in one of the main results in Shannon's paper, his [channel coding theorem](https://en.wikipedia.org/wiki/Noisy-channel_coding_theorem).